<a href="https://colab.research.google.com/github/fiapdatanalytics/tech-challenge4/blob/main/pipeline_modelo_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import re
import json
import requests
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    StandardScaler,
    OrdinalEncoder,
    OneHotEncoder,
    LabelEncoder
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

#funcoes utilizadas

def renomear_colunas(df: pd.DataFrame, mapa_renomeacao: dict = None) -> pd.DataFrame:
    """
    Renomeia as colunas de um DataFrame utilizando um dicionário fornecido
    ou aplicando um padrão de formatação (minúsculas, espaços por '_').
    Retorna o DataFrame com as colunas renomeadas.
    """
    colunas_originais = df.columns.tolist()
    df_modificado = df.copy()

    if mapa_renomeacao:
        if any(col in mapa_renomeacao for col in colunas_originais):
            df_modificado.rename(columns=mapa_renomeacao, inplace=True)
        else:
            print('Não foi possível realizar a alteração dos nomes das colunas. Rever arquivo de entrada.')
    else:
        print('Não foi possível realizar a alteração dos nomes das colunas. Rever arquivo de entrada.')

    return df_modificado

def transformar_valores_string(df: pd.DataFrame, coluna: str, mapa_transformacao: dict):
    """
    Transforma os valores de uma coluna do tipo string para string utilizando um mapeamento.
    Modifica o DataFrame inplace.
    """
    if coluna not in df.columns:
        print(f"Erro: A coluna '{coluna}' não existe no DataFrame.")
        return

    if df[coluna].dtype != 'object':
        print(f"Erro: A coluna '{coluna}' não é do tipo 'object'. Nenhuma transformação será aplicada.")
        return

    valores_unicos_na_coluna = df[coluna].dropna().unique()
    valores_nao_mapeados = [valor for valor in valores_unicos_na_coluna if valor not in mapa_transformacao]

    if valores_nao_mapeados:
        print(f"Erro: Os seguintes valores únicos na coluna '{coluna}' não foram encontrados no mapeamento:")
        print(valores_nao_mapeados)
        print("A transformação não será realizada pois nem todos os valores possuem um mapeamento.")
        return

    df[coluna] = df[coluna].map(mapa_transformacao)
    print(f"Coluna '{coluna}' transformada com sucesso.")


def ler_json_de_url(url: str) -> dict:
    """
    Lê um arquivo JSON de uma URL fornecida.
    """
    try:
        resposta = requests.get(url)
        resposta.raise_for_status()
        dict_dados = resposta.json()
        return dict_dados
    except Exception as e:
        print(f"Erro ao ler JSON de {url}: {e}")
        return None


class MtransGrouper(BaseEstimator, TransformerMixin):
    """Agrupa as categorias raras de 'mtrans'."""
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_array = np.asarray(X)
        X_series = pd.Series(X_array.flatten(), name='mtrans')
        mtrans_agrupado = X_series.replace(
            ['moto', 'bicicleta', 'caminhando'], 'outros'
        )
        return mtrans_agrupado.values.reshape(-1, 1)


class CalcGrouper(BaseEstimator, TransformerMixin):
    """Agrupa as categorias raras de 'calc'."""
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_array = np.asarray(X)
        X_series = pd.Series(X_array.flatten(), name='calc')
        sempre_freq = X_series.replace(
            'sempre', 'frequentemente'
        )
        return sempre_freq.values.reshape(-1, 1)

class RoundingTransformer(BaseEstimator, TransformerMixin):
    """Arredonda os dados sintéticos (ex: 2.45 -> 2)"""
    def fit(self, X, y=None):
        return self

    def transform(self, X, **kwargs):
        return np.round(X).astype(int)


#etapa - importacao de dados

df = pd.read_csv('https://raw.githubusercontent.com/fiapdatanalytics/tech-challenge4/refs/heads/main/data/Obesity.csv')
mapa_colunas = ler_json_de_url('https://raw.githubusercontent.com/fiapdatanalytics/tech-challenge4/refs/heads/main/data/mapa_colunas.json')
mapa_valores_colunas = ler_json_de_url('https://raw.githubusercontent.com/fiapdatanalytics/tech-challenge4/refs/heads/main/data/mapa_valores_colunas.json')

print("DataFrame e mapas JSON carregados com sucesso.")

df_processado = renomear_colunas(df, mapa_colunas)

#etapa - pre-processamento

# Transformar valores de string
if mapa_valores_colunas:
    if 'mapeamento_classificacao_peso_corporal' in mapa_valores_colunas:
        transformar_valores_string(df_processado, 'classificacao_peso_corporal' , mapa_valores_colunas['mapeamento_classificacao_peso_corporal']['valores_novos_classificacao_peso_corporal'] )
    if 'mapeamento_mtrans' in mapa_valores_colunas:
         transformar_valores_string(df_processado,'mtrans' , mapa_valores_colunas['mapeamento_mtrans']['valores_novos_mtrans'])
    if 'mapeamento_frequencia' in mapa_valores_colunas:
        transformar_valores_string(df_processado,'caec' , mapa_valores_colunas['mapeamento_frequencia']['valores_novos_frequencia'])
        transformar_valores_string(df_processado,'calc' , mapa_valores_colunas['mapeamento_frequencia']['valores_novos_frequencia'])
    if 'mapeamento_genero' in mapa_valores_colunas:
        transformar_valores_string(df_processado,'genero' , mapa_valores_colunas['mapeamento_genero']['transformacao_genero'])
    if 'mapeamento_sim_nao' in mapa_valores_colunas:
        colunas_sim_nao = ['historico_familiar', 'favc', 'fumante', 'scc']
        for coluna in colunas_sim_nao:
            transformar_valores_string(df_processado, coluna , mapa_valores_colunas['mapeamento_sim_nao']['transformacao_sim_nao'])

print("Pré-processamentos iniciais aplicados e df_processado criado.")

coluna_alvo = 'classificacao_peso_corporal'

# Remover variaveis com vazamento ou risco extremo e colunas originais float
variaveis_a_remover_de_X_antes_do_pipeline = [
    'peso', 'altura', 'fumante',
]

X = df_processado.drop(columns=[coluna_alvo] + variaveis_a_remover_de_X_antes_do_pipeline)
y = df_processado[coluna_alvo]

# Codificar o Alvo (y) de texto para números
le = LabelEncoder()
y_codificada = le.fit_transform(y)

# Separar dados em treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, y_codificada,
    test_size=0.2,
    random_state=42,
    stratify=y_codificada
)

print("Dados preparados: X, y definidos, y codificado e dividido em treino/teste.")

variaveis_continuas = ['idade']
variaveis_bin_nominal = ['genero', 'historico_familiar', 'favc', 'scc']
variaveis_multi_nominal = ['mtrans']
variaveis_clean_ordenadas = ['caec']
variaveis_para_arredondar_e_codificar = ['fcvc', 'ncp', 'ch20', 'faf', 'tue']

pipeline_continua = Pipeline(steps=[
    ('scaler', StandardScaler())
])

pipeline_arrredonamento_ordenacao = Pipeline(steps=[
    ('rounder', RoundingTransformer()),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

pipeline_ordenada_limpa = Pipeline(steps=[
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

pipeline_calc = Pipeline(steps=[
    ('grouper', CalcGrouper()),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

pipeline_nominal_bin = Pipeline(steps=[
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

pipeline_nominal_multi = Pipeline(steps=[
    ('grouper', MtransGrouper()),
    ('encoder', OneHotEncoder(handle_unknown='ignore', drop='first'))
])

preprocessador = ColumnTransformer(
    transformers=[
        ('cont', pipeline_continua, variaveis_continuas),
        ('arredondada_ord', pipeline_arrredonamento_ordenacao, variaveis_para_arredondar_e_codificar),
        ('ord_limpa', pipeline_ordenada_limpa, variaveis_clean_ordenadas),
        ('calc_pipe', pipeline_calc, ['calc']),
        ('nom_bin', pipeline_nominal_bin, variaveis_bin_nominal),
        ('nom_multi', pipeline_nominal_multi, variaveis_multi_nominal)
    ],
    remainder='drop'
)

modelo_xgb = XGBClassifier(
    objective='multi:softmax',
    num_class=7,
    n_jobs=-1,
    eval_metric='mlogloss',
    use_label_encoder=False,
    learning_rate= 0.2,
    max_depth= 7,
    n_estimators= 200,
    subsample= 0.8,
    random_state=42
)

pipeline_completa_xgb = Pipeline(steps=[
    ('preprocessor', preprocessador),
    ('model', modelo_xgb)
])

print("Pipelines de pré-processamento e o modelo XGBoost definidos e combinados no pipeline completo.")

#etapa - treinameto
print("Iniciando o treinamento do pipeline XGBoost completo...")

pipeline_completa_xgb.fit(X_treino, y_treino)

y_prev_xgb = pipeline_completa_xgb.predict(X_teste)
xgb_acuracia = accuracy_score(y_teste, y_prev_xgb)
print(f"Acurácia do Pipeline XGBoost: {xgb_acuracia * 100:.2f}%")

nomes_classes = le.classes_
print("\nRelatório de Classificação:")
print(classification_report(y_teste, y_prev_xgb, target_names=nomes_classes))

print("\nPipeline de pré-processamento e modelagem concluído!")

print("Salvando artefatos...")
joblib.dump(pipeline_completa_xgb, 'pipeline_obesidade_completo.joblib')
joblib.dump(le, 'label_encoder_xgb.joblib')
print("Artefatos salvos com sucesso!")

DataFrame e mapas JSON carregados com sucesso.
Coluna 'classificacao_peso_corporal' transformada com sucesso.
Coluna 'mtrans' transformada com sucesso.
Coluna 'caec' transformada com sucesso.
Coluna 'calc' transformada com sucesso.
Coluna 'genero' transformada com sucesso.
Coluna 'historico_familiar' transformada com sucesso.
Coluna 'favc' transformada com sucesso.
Coluna 'fumante' transformada com sucesso.
Coluna 'scc' transformada com sucesso.
Pré-processamentos iniciais aplicados e df_processado criado.
Dados preparados: X, y definidos, y codificado e dividido em treino/teste.
Pipelines de pré-processamento e o modelo XGBoost definidos e combinados no pipeline completo.
Iniciando o treinamento do pipeline XGBoost completo...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [14:54:36] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Acurácia do Pipeline XGBoost: 82.98%

Relatório de Classificação:
                   precision    recall  f1-score   support

 obesidade_tipo_1       0.78      0.86      0.82        70
 obesidade_tipo_2       0.89      0.92      0.90        60
 obesidade_tipo_3       0.98      1.00      0.99        65
peso_insuficiente       0.84      0.89      0.86        54
      peso_normal       0.68      0.72      0.70        58
 sobrepeso_tipo_1       0.84      0.72      0.78        58
 sobrepeso_tipo_2       0.80      0.67      0.73        58

         accuracy                           0.83       423
        macro avg       0.83      0.83      0.83       423
     weighted avg       0.83      0.83      0.83       423


Pipeline de pré-processamento e modelagem concluído!
Salvando artefatos...
Artefatos salvos com sucesso!
